# Remote upload of VR media


To remotely upload data and files, we will use the REST API provided by the iScience server.  There are several steps involved with uploading then connecting the upload with an experiment. The relationship between these is described as:

Experiment -- [1-many]-- ExperimentResult -- [1 to 1] -- VR media

The steps are as follows:

1. Create a Session (username/password) to get a token
1. Create VR media object with file upload
1. Create an Experiment (or use an existing experiment)
2. Create an Experiment Result linked to the experiment containing the VR media

## Create a plot directory

In [13]:
import os
DATA_DIR = os.path.join(os.getcwd(), 'data')
if not os.path.exists(DATA_DIR):
    os.mkdir(DATA_DIR)
print(DATA_DIR)

/home/jovyan/work/data


## Create an authenticated session

In [1]:
username=input('What is your username? ')

What is your username? demo


In [2]:
password=input('What is your password (this will show in plain text)? ')

What is your password (this will show in plain text)? iScience2019


In [3]:
import requests
from requests.auth import HTTPBasicAuth
import json
from urllib.parse import urlparse, urljoin
import petl as etl
import certifi
import urllib3
# Using SSL so require certification but no access to certs so turn off the warning
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Enter URL for experiments API
exptbaseurl = 'https://agri-ml.it.csiro.au:8443'

# Get an access token - replace user and password with actual values
serviceurl = exptbaseurl + '/api-token-auth/'
data = dict(username=username, password=password)

auth = requests.post(serviceurl, verify=False, data=data)

if auth.status_code != 200:
    print("Could not successfully access REST API: " + serviceurl)
else:
    token = 'Token ' + auth.json()['token']
    print(token)

Token 37a561a3c66eb68eb95e8101905ce49a087be3ef


## Upload VR media

1. Upload the VR video and optionally, a thumbnail image to this server (data directory) at https://agri-ml.it.csiro.au:8888/tree? by clicking on upload
  1. Max file size is currently set to 100MB (limited by dev server)
  1. Thumbnail image dimensions of 300px x 400px png image is preferred
1. Create an entry for the VR file in the database


In [7]:
vr_url = exptbaseurl + '/vrmodules/'
headers = {'Authorization': token}

# Describe the file to upload
name = "AIM VR Video"
description = "VR media upload remotely"
filename = "outputMain.mp4" # Relative path to file
vrtype = "Video" # Choice of Video, Image or Shapes

# Set data details for VR media with file upload
data = {"id":"0", 
        "name": name, 
        "description": description, 
        "filename": filename, 
        "uri": "",
        "stereo": "true", 
        "autopanoff": "false", 
        "width": "100%",
        "height": "400", 
        "type": vrtype, 
        "markup": " `<a-scene><a-sky src=\"" + filename + "\" rotation=\"0 -130 0\"></a-sky></a-scene>`"}
files = {'vrfile': open(os.path.join(DATA_DIR, filename),'rb')}

# Send to the server
try:
    r = requests.post(vr_url, verify=False, headers=headers, files=files, data=data)
    r.raise_for_status()
    response = r.json()
    print("Successfully created the new VR: id = " + str(response["id"]))
    vr_id = response["id"] # the new ID assigned to the VR upload
except requests.exceptions.HTTPError as e:
    print(e)   

Successfully created the new VR: id = 21


## Check the VR module in the database

In [8]:
vr_url = exptbaseurl + '/vrmodules/' + str(vr_id)
headers = {'Authorization': token, 'Content-type': 'application/json'}

try:
    r = requests.get(vr_url, verify=False, headers=headers)
    r.raise_for_status()
    response = r.json()
    print("Printing content of VR: id = " + str(vr_id))
    print("***********")
    for key in response.keys():
        print(key + ": " + str(response[key]))
except requests.exceptions.HTTPError as e:
    print(e) 

Printing content of VR: id = 21
***********
id: 21
vr_results: []
name: AIM VR Video
description: VR media upload remotely
filename: outputMain.mp4
uri: 
width: 100%
height: 400
stereo: True
autopanoff: False
type: Video
markup: `<a-scene><a-sky src="outputMain.mp4" rotation="0 -130 0"></a-sky></a-scene>`
vrfile: https://agri-ml.it.csiro.au:8443/media/outputMain_gYEMgN4.mp4


## Get an existing Experiment
Get the required Experiment ID from the iScience portal.

In [10]:
# Set required Experiment ID
expt_id = 12 

# Retrieve experiment to check (optional step)
expt_url = exptbaseurl + '/experiments/' + str(expt_id)
try:
    r = requests.get(expt_url, verify=False, headers=headers)
    r.raise_for_status()
    response = r.json()
    print("Retrieved Experiment: id = " + str(expt_id))
    print("***********")
    print("Name: " + str(response["name"]))
except requests.exceptions.HTTPError as e:
    print(e) 

Retrieved Experiment: id = 12
***********
Name: Experiment for uploaded VR


### Create Experiment Result with VR Media

In [12]:
# Set URl
expt_results_url = exptbaseurl + '/experiment_results/'

# Set Data details to upload
data = {"id": "0", 
        "description": "AIM video of powder coating", 
        "result_type": "VR",
        "experiment": str(expt_id), 
        "vrmodule": str(vr_id)}

# Optional, add a thumbnail - EXACT filename (case sensitive)
files = {"thumbnail" : open(os.path.join(DATA_DIR, "AIM_video.PNG"), "rb")}
headers = {'Authorization': token}

try:
    r = requests.post(expt_results_url, verify=False, headers=headers, files=files, data=data)
    r.raise_for_status()
    response = r.json()
    print("Successfully linked the VR to Experiment: id = " + str(response["id"]))
    expt_result_id = response["id"] # the new ID assigned to the Experiment Result
except requests.exceptions.HTTPError as e:
    print(e)   

Successfully linked the VR to Experiment: id = 29


## Create a new Experiment to link to
If you need to create the experiment first, there are some more data to provide.

*It is important to ensure the type and status match the expected values as they are lookup values*

In [36]:
data = {
    "type": {
        "id": 2,
        "name": "Spectroscopy"
    },
    "status": {
        "id": 1,
        "name": "Active"
    },
    "name": "Experiment for uploaded VR",
    "number": "001",
    "accession": "accessnumber",
    "comments": "comments here",
    "samplecode": "",
    "summary": "CREATED from Jupyter Notebook",
    "uident": "tst001",
    "results": []
}

#### Create the new experiment

In [37]:
expt_url = exptbaseurl + '/experiments/'
headers = {'Authorization': token, 'Content-type': 'application/json'}
try:
    r = requests.post(expt_url, verify=False, headers=headers, json=data)
    r.raise_for_status()
    response = r.json()
    print("Successfully created the new Experiment: id=" + str(response["id"]) )
except requests.exceptions.HTTPError as e:
    print(e)  

Successfully created the new Experiment: id=6


#### Add the VR to the new experiment
A copy of the code above

In [12]:
# Set URl
expt_results_url = exptbaseurl + '/experiment_results/'

# Set Data details to upload
data = {"id": "0", 
        "description": "AIM video of powder coating", 
        "result_type": "VR",
        "experiment": str(expt_id), 
        "vrmodule": str(vr_id)}

# Optional, add a thumbnail - EXACT filename (case sensitive)
files = {"thumbnail" : open(os.path.join(DATA_DIR, "AIM_video.PNG"), "rb")}
headers = {'Authorization': token}

try:
    r = requests.post(expt_results_url, verify=False, headers=headers, files=files, data=data)
    r.raise_for_status()
    response = r.json()
    print("Successfully linked the VR to Experiment: id = " + str(response["id"]))
    expt_result_id = response["id"] # the new ID assigned to the Experiment Result
except requests.exceptions.HTTPError as e:
    print(e)   

Successfully linked the VR to Experiment: id = 29


**When all done, login to the portal at https://agri-ml.it.csiro.au/ and view your new results**